In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
import os
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
#os.environ["TF_CPP_VMODULE"] = "gpu_process_state=10,gpu_cudamallocasync_allocator=10"
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        # allowing gpu memory growth
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)


1 Physical GPUs, 1 Logical GPU


2021-12-18 17:31:14.005217: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-18 17:31:14.103610: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-18 17:31:14.103848: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-18 17:31:14.105955: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=3)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


In [5]:
from sklearn.model_selection import train_test_split
dataset = pd.read_csv('Twitter_Data(Relabeled).csv')
dataset.columns=['clean_text','category']
dataset['category']=pd.to_numeric(dataset['category'],errors='coerce')
dataset=dataset.dropna(subset=['category'])
dataset['category']=dataset['category'].astype(int)
train , test =train_test_split(dataset,train_size=0.7)
train.head(10)

,clean_text,category
35491,best tweet,2
133028,even the based study shows the true scenario t...,0
123581,bro after this victory will see different modi...,1
25098,lots development projects too below explain so...,1
50890,vajpayee made india stronger with nuclear powe...,1
33031,news nation conclave what subramanian swamy sa...,1
153603,watch how modi punctures the claims made chidu...,2
24077,khan said shadows war were still hovering over...,2
104882,watch you want mazboot strong government majbo...,2
12716,sir have trying two times make voterid card bu...,2


In [6]:
test.head()

,clean_text,category
116540,bigoted biased left inclined anti hindu anti m...,2
62821,because back then there was advertiser like modi,1
107765,narendra modi will again our prime minister 2019,1
102248,madam your victory confirming now positive env...,2
101180,only new revelation interview that not apologe...,2


In [7]:
InputExample(guid=None,
             text_a= "Hello, world",
             text_b=None,
             label=1.0)

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1.0)

In [8]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'clean_text', 
                                                                           'category')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        try:
          input_dict = tokenizer.encode_plus(
              e.text_a,
              add_special_tokens=True,
              max_length=max_length, # truncates if len(s) > max_length
              return_token_type_ids=True,
              return_attention_mask=True,
              pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
              truncation=True
          )

          input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
              input_dict["token_type_ids"], input_dict['attention_mask'])

          features.append(
              InputFeatures(
                  input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
              )
          )
        except:
          print("error produced by:")
          print(e)

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'clean_text'
LABEL_COLUMN = 'category'

In [9]:
len(dataset)

162973

In [10]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(16).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(16)

/home/rafxar/Proyecto-Final-De-Carrera-1/bert-env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


error produced by:
InputExample(guid=None, text_a=nan, text_b=None, label=2)
error produced by:
InputExample(guid=None, text_a=nan, text_b=None, label=1)
error produced by:
InputExample(guid=None, text_a=nan, text_b=None, label=1)
error produced by:
InputExample(guid=None, text_a=nan, text_b=None, label=0)


In [11]:
##Using 3e-5 as our learning rate
from sklearn import metrics
checkpoint_path = "training_1/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
try:
    model.load_weights(checkpoint_path)
except:
    print("No checkpoint found")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=5, validation_data=validation_data,
          callbacks=[cp_callback])


No checkpoint found
Epoch 1/5
  14260/Unknown - 5662s 396ms/step - loss: 0.1330 - accuracy: 0.9553
Epoch 00001: saving model to training_1/cp.ckpt
14260/14260 [==============================] - 6064s 424ms/step - loss: 0.1330 - accuracy: 0.9553 - val_loss: 0.0799 - val_accuracy: 0.9747
Epoch 2/5
14260/14260 [==============================] - ETA: 0s - loss: 0.0444 - accuracy: 0.9864
Epoch 00002: saving model to training_1/cp.ckpt
14260/14260 [==============================] - 5587s 392ms/step - loss: 0.0444 - accuracy: 0.9864 - val_loss: 0.0760 - val_accuracy: 0.9794
Epoch 3/5
14260/14260 [==============================] - ETA: 0s - loss: 0.0280 - accuracy: 0.9919
Epoch 00003: saving model to training_1/cp.ckpt
14260/14260 [==============================] - 5914s 415ms/step - loss: 0.0280 - accuracy: 0.9919 - val_loss: 0.1290 - val_accuracy: 0.9770
Epoch 4/5
14260/14260 [==============================] - ETA: 0s - loss: 0.0220 - accuracy: 0.9944
Epoch 00004: saving model to training_1/

In [11]:
#Using 5e-5 as our learning rate
#3e-4, 1e-4, 5e-5
from sklearn import metrics
checkpoint_path = "training_1/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
try:
    model.load_weights(checkpoint_path)
except:
    print("No checkpoint found")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08, clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=5, validation_data=validation_data,
          callbacks=[cp_callback])

No checkpoint found
Epoch 1/5
  14260/Unknown - 5394s 377ms/step - loss: 0.1523 - accuracy: 0.9507
Epoch 00001: saving model to training_1/cp.ckpt
14260/14260 [==============================] - 5787s 405ms/step - loss: 0.1523 - accuracy: 0.9507 - val_loss: 0.0954 - val_accuracy: 0.9689
Epoch 2/5
14260/14260 [==============================] - ETA: 0s - loss: 0.0662 - accuracy: 0.9812
Epoch 00002: saving model to training_1/cp.ckpt
14260/14260 [==============================] - 5632s 395ms/step - loss: 0.0662 - accuracy: 0.9812 - val_loss: 0.1312 - val_accuracy: 0.9687
Epoch 3/5
14260/14260 [==============================] - ETA: 0s - loss: 0.0482 - accuracy: 0.9872
Epoch 00003: saving model to training_1/cp.ckpt
14260/14260 [==============================] - 5475s 384ms/step - loss: 0.0482 - accuracy: 0.9872 - val_loss: 0.1039 - val_accuracy: 0.9727
Epoch 4/5
14260/14260 [==============================] - ETA: 0s - loss: 0.0388 - accuracy: 0.9903
Epoch 00004: saving model to training_1/

In [12]:
#using 3e-4 as our learning rate
from sklearn import metrics
checkpoint_path = "training_1/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
try:
    model.load_weights(checkpoint_path)
except:
    print("No checkpoint found")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4, epsilon=1e-08, clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=5, validation_data=validation_data,
          callbacks=[cp_callback])

No checkpoint found
Epoch 1/5
  14260/Unknown - 5515s 386ms/step - loss: 1.1093 - accuracy: 0.4073
Epoch 00001: saving model to training_1/cp.ckpt
14260/14260 [==============================] - 5903s 413ms/step - loss: 1.1093 - accuracy: 0.4073 - val_loss: 1.0768 - val_accuracy: 0.4392
Epoch 2/5
   63/14260 [..............................] - ETA: 1:31:01 - loss: 1.0814 - accuracy: 0.4107

KeyboardInterrupt: 

In [12]:
pred_tweets = ['president is such a lazy person, does not work at all', 'pinapple pizza may just be the worst dish ever created',
               'last night episode was the most engaging in the last two seasons', 'pasta with no sauce and Fettuccine Alfredo']
tf_batch=tokenizer(pred_tweets, max_length=128, padding=True, truncation=True,return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions=tf.nn.softmax(tf_outputs[0],axis=-1)
labels = ['Negative','Neutral','Positive']
label=tf.argmax(tf_predictions,axis=-1)
label=label.numpy()
for i in range(len(pred_tweets)):
    print(pred_tweets[i], ": \n", labels[label[i]])


president is such a lazy person, does not work at all : 
 Negative
pinapple pizza may just be the worst dish ever created : 
 Negative
last night episode was the most engaging in the last two seasons : 
 Positive
pasta with no sauce and Fettuccine Alfredo : 
 Neutral
